In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.io import loadmat
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from astropy.stats import circvar
from astropy import units as u

In [2]:
def replace_area_name(df, area_key):
# df = dataframe containing an area array column
# area_key = string

    df[area_key] = df[area_key].replace(1, 'V1')
    df[area_key] = df[area_key].replace(2, 'LM')
    df[area_key] = df[area_key].replace(3, 'LI')
    
    return df

- areacode, vis_driven, well_fit  
- ori_perc  
- OSI, R2, width, SNR